In [2]:
!pip install langchain langchain-core langchain-community langchain-groq

In [3]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.linkedin.com/jobs/view/4351214762/?alternateChannel=search&eBP=BUDGET_EXHAUSTED_JOB&refId=9OD1On4Xj6g0d6r9K%2B%2BEZg%3D%3D&trackingId=02Mt9mxXQmdL2TESgGNujg%3D%3D")
job_data = loader.load()
type(job_data)


USER_AGENT environment variable not set, consider setting it to identify your requests.


list

In [4]:
title = job_data[0].metadata.get("title")
description = job_data[0].metadata.get("description")
content = job_data[0].page_content

In [5]:
import re

def clean_text(text: str) -> str:
    # Replace newlines & tabs with space
    text = re.sub(r"[\n\t\r]+", " ", text)

    # Collapse multiple spaces into one
    text = re.sub(r"\s{2,}", " ", text)

    # Strip leading/trailing space
    return text.strip()


In [6]:
content = clean_text(content)
content

'Jupiter hiring Decision Science Intern in Bengaluru, Karnataka, India | LinkedIn Skip to main content LinkedIn Decision Science Intern in Mumbai Expand search This button displays the currently selected search type. When expanded it provides a list of search options that will switch the search inputs to match the current selection. Jobs People Learning Clear text Clear text Clear text Clear text Clear text Sign in Join for free Decision Science Intern Jupiter Bengaluru, Karnataka, India Apply Decision Science Intern Jupiter Bengaluru, Karnataka, India 22 hours ago Over 200 applicants See who Jupiter has hired for this role Apply Save Report this job Use AI to assess how you fit Get AI-powered advice on this job and more exclusive features. Am I a good fit for this job? Tailor my resume Sign in to access AI-powered advices Sign in Welcome back Email or phone Password Show Forgot password? Sign in or By clicking Continue to join or sign in, you agree to LinkedIn’s User Agreement, Privac

In [7]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

load_dotenv()
groq = os.getenv("GROQ")

llm = ChatGroq(
    groq_api_key=groq,
    model="llama-3.1-8b-instant",
    temperature=0.7
)

llm

c:\Users\Tatkare\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000024811E95160>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000024811E95E80>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [8]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

prompt_template = PromptTemplate.from_template(
    "You are helpful expert assistant, get the incharge of job posting of hiring team mention in this job postion content: {content}."
    "Provide the result in clean short JSON strict format"
)
parser = JsonOutputParser()

chain = prompt_template | llm | parser
res = chain.invoke({
    "content": content
})
res

{'JobTitle': 'Decision Science Intern',
 'Company': 'Jupiter',
 'Location': 'Bengaluru, Karnataka, India',
 'Duration': '3-6 months',
 'Requirements': ['Recently completed a degree in Data Science, Statistics, Economics, Engineering, or related field',
  'Strong analytical and problem-solving skills',
  'Proficiency in SQL and Python',
  'Familiarity with data visualization tools (Tableau, Power BI, or similar)',
  'Excellent communication and presentation skills',
  'Curiosity, attention to detail, and a passion for working with data'],
 'Responsibilities': ['Analyze large datasets to identify patterns, trends, and opportunities',
  'Build dashboards, reports, and visualizations to communicate findings',
  'Assist in designing and running experiments (A/B testing, user behavior analysis, etc.)',
  'Support decision-making by providing data-driven recommendations',
  'Collaborate with product managers, engineers, and business stakeholders',
  'Document methodologies and share insights 

In [9]:
prompt_t = PromptTemplate.from_template(
    """
You are an expert career assistant.

Generate a professional job application email **in JSON format** with exactly two keys:
1. "subject" → short and professional subject line.
2. "body" → email body text.

Rules for the email body:
- Structure the email in **4 paragraphs**, 1–2 lines per paragraph, with a line space between paragraphs.
- Start with a greeting addressing the hiring manager's name from {res} (if unavailable, use "Hi Hiring Manager").
- End with a professional closing (e.g., "Best regards, Vaibhav Tatkare").
- Mention your **resume link** naturally: https://drive.google.com/file/d/1lzFyv7BKdrw7y0hx9Lobo1obLMPASaiu/view?usp=sharing
- Mention your **GitHub link** naturally: https://github.com/VaibhavT04
- Show your **knowledge of their company and the role**, emphasizing alignment and why you are best suited.
- Keep it concise, professional, confident, and eager.
- Candidate profile: 4th-year Computer Engineering student, CGPA 9.23, hands-on AI/ML and Full-Stack experience, production-ready projects on GitHub.
- Do not exceed 120 words.
- Output **only valid JSON**, no explanations, no markdown.

Hiring Manager Info:
{res}

Job Details:
{content}
"""
)

final_chain = prompt_t | llm | parser
mail = final_chain.invoke({
    "res": res,
    "content": content
})
mail

{'subject': 'Application for Decision Science Intern at Jupiter',
 'body': "Hi Aanvee Verma,\n\nI am thrilled to apply for the Decision Science Intern role at Jupiter, as advertised on LinkedIn. With a strong background in data analysis, machine learning, and full-stack development, I believe I would be an excellent fit for this position.\n\nI am particularly drawn to Jupiter's mission of empowering individuals with a customer-centric experience in personal finance. My passion for working with data and driving business growth aligns with the company's vision.\n\nI have attached my resume for your review. You can also find more information about my projects on GitHub (https://github.com/VaibhavT04).\n\nThank you for considering my application. I look forward to discussing this opportunity further.\n\nBest regards,\nVaibhav Tatkare"}

In [ ]:
import smtplib
from email.mime.text import MIMEText

gmail_pass = os.getenv("GMAIL_PASS")

# Gmail credentials
gmail_user = "vaibhavtatkare2004@gmail.com"
gmail_pass =  gmail_pass

# Receiver
to_email = "vaibhavtatkare04@gmail.com"

# Create message
msg = MIMEText(mail['body'])
msg['Subject'] = mail['subject']
msg['From'] = gmail_user
msg['To'] = to_email

# Send email
with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
    server.login(gmail_user, gmail_pass)
    server.send_message(msg)

print("Email sent successfully!")


SyntaxError: invalid syntax (4263483455.py, line 8)

wwhp ttiw gcdv rcyk
